In [1]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [4]:
train_dataset = torchvision.datasets.CIFAR10(
    root = './data',
    train = True,
    transform = transform,
    download = True
)

test_dataset = torchvision.datasets.CIFAR10(
    root = './data',
    train = False,
    transform = transform,
    download = False
)

100%|██████████| 170498071/170498071 [00:01<00:00, 106316045.67it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [5]:
batch_size = 4
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    shuffle = True,
    batch_size = batch_size
)

test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    shuffle = False,
    batch_size = batch_size
)

In [6]:
class ConvolutionalNeuralNetwork(nn.Module):
  def __init__(self):
    super(ConvolutionalNeuralNetwork, self).__init__()
    self.relu = nn.ReLU()

    self.convolutional_1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 5)
    self.pool_1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.convolutional_2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5)
    self.pool_2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.linear_1 = nn.Linear(16 * 5 * 5, 120)
    self.linear_2 = nn.Linear(120, 84)
    self.linear_3 = nn.Linear(84, 10)

  def forward(self, x):
    self.output = self.convolutional_1(x)
    self.output = self.relu(self.output)
    self.output = self.pool_1(self.output)

    self.output = self.convolutional_2(self.output)
    self.output = self.relu(self.output)
    self.output = self.pool_2(self.output)

    self.output = self.output.view(-1, 16 * 5 * 5)
    self.output = self.linear_1(self.output)
    self.output = self.relu(self.output)

    self.output = self.linear_2(self.output)
    self.output = self.relu(self.output)

    self.output = self.linear_3(self.output)
    return self.output

model = ConvolutionalNeuralNetwork().to(device)
print(model)

ConvolutionalNeuralNetwork(
  (relu): ReLU()
  (convolutional_1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convolutional_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_1): Linear(in_features=400, out_features=120, bias=True)
  (linear_2): Linear(in_features=120, out_features=84, bias=True)
  (linear_3): Linear(in_features=84, out_features=10, bias=True)
)


In [7]:
learning_rate = 1e-3
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
epochs = 4
for epoch in range(epochs):
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = loss_function(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if epoch % 1 == 0:
    print(f'epoch : {epoch + 1}/{epochs} | loss : {loss.item() : .4f}')

with torch.no_grad():
  correct_predictions = 0
  total_samples = 0

  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, predictions = torch.max(outputs, 1)
    total_samples += labels.shape[0]
    correct_predictions += (predictions == labels).sum().item()

  accuracy = 100 * correct_predictions / total_samples
  print(f'accuracy : {accuracy}')

epoch : 1/4 | loss :  2.5612
epoch : 2/4 | loss :  0.4576
epoch : 3/4 | loss :  1.2489
epoch : 4/4 | loss :  1.5028
accuracy : 59.03
